In [ ]:
import json
import os
from utils.data_dirs import data_dirs
from utils.filter_road import filter_road

In [ ]:
d = data_dirs()
print(d.raw_road)
print(d.processed_road)
print(d.raw_map)
print(d.processed_map)

In [ ]:
raw_expansion = os.path.join(d.raw_map, 'expansion')
processed_expansion = os.path.join(d.processed_map, 'expansion')

In [ ]:
_tokens = []
for e in os.listdir(raw_expansion):
    with open(os.path.join(raw_expansion, e), "r") as f:
        e_json = json.load(f)
    
    tokens: "Set[str]" = set()
        
    for k, value in e_json.items():
        if type(value) == list:
            tokens.update({v["token"] for v in value if type(v) == dict})
        elif type(value) == dict:
            tokens.update(value.keys())
    
    if not os.path.exists(processed_expansion):
        os.makedirs(processed_expansion)
    
    with open(os.path.join(processed_expansion, e[:-len(".json")] + "-tokens.json"), "w") as f:
        json.dump([*tokens], f)
    
    _tokens.append(tokens)

for t1 in _tokens:
    for t2 in _tokens:
        if t1 == t2: continue
        print(t1 & t2)

In [ ]:
for tokens_f in os.listdir(processed_expansion):
    with open(os.path.join(processed_expansion, tokens_f), "r") as f:
        tokens = {*json.load(f)}
    
    region = tokens_f[:-len("-tokens.json")]
    print(region)
    filtered = filter_road(d.raw_road, tokens)
    for k, v in filtered.items():
        print(k, len(v))
        base = os.path.join(d.processed_road, region)
        if not os.path.exists(base):
            os.makedirs(base)
        with open(os.path.join(base, k + ".json"), "w") as f:
            json.dump(v, f)
    